In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('rheeza').getOrCreate()

23/09/02 09:53:35 WARN Utils: Your hostname, Yinkas-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.51 instead (on interface en0)
23/09/02 09:53:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/02 09:53:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/02 09:53:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# Understand the Dataset

In [11]:
#reading the json
trials_df = spark.read.json('/Users/yinka/PycharmProjects/pythonProject5/venv/dataset.json', multiLine=True)

In [12]:
trials_df.show(5)

+----------------+--------------------+---------+-----------------+-------------------+------------------------------+--------------------+
|ageofparticipant|           clinician|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|              result|
+----------------+--------------------+---------+-----------------+-------------------+------------------------------+--------------------+
|              19|{Ontario, Saul, t...|  Placebo|    1619827200000|      1617235200000|                            52|{BP normalized, r...|
|              14|{Ontario, Saul, n...| Naproxen|    1619827200000|      1617235200000|                            78|    {Follow-up, N/A}|
|              17|{Ontario, Saul, n...|  Placebo|    1619827200000|      1617235200000|                            14|    {Follow-up, N/A}|
|              18|{Ontario, Will, n...| Naproxen|    1619827200000|      1617235200000|                            14|{BP normalized, N/A}|
|              17|{O

In [8]:
trials_df.dtypes

[('_corrupt_record', 'string'),
 ('ageofparticipant', 'bigint'),
 ('clinician', 'struct<branch:string,name:string,role:string>'),
 ('drug_used', 'string'),
 ('experimentenddate', 'string'),
 ('experimentstartdate', 'string'),
 ('noofhourspassedatfirstreaction', 'bigint'),
 ('result', 'struct<conclusion:string,sideeffectsonparticipant:string>')]

In [17]:
trials_df.printSchema()

root
 |-- ageofparticipant: long (nullable = true)
 |-- clinician: struct (nullable = true)
 |    |-- branch: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- role: string (nullable = true)
 |-- drug_used: string (nullable = true)
 |-- experimentenddate: string (nullable = true)
 |-- experimentstartdate: string (nullable = true)
 |-- noofhourspassedatfirstreaction: long (nullable = true)
 |-- result: struct (nullable = true)
 |    |-- conclusion: string (nullable = true)
 |    |-- sideeffectsonparticipant: string (nullable = true)



In [13]:
trials_df.columns

['ageofparticipant',
 'clinician',
 'drug_used',
 'experimentenddate',
 'experimentstartdate',
 'noofhourspassedatfirstreaction',
 'result']

In [15]:
columns = [
 'ageofparticipant',
  'clinician.branch', 'clinician.name', 'clinician.role',
 'drug_used',
 'experimentenddate',
 'experimentstartdate',
 'noofhourspassedatfirstreaction',
 'result.conclusion', 'result.sideeffectsonparticipant']

In [16]:
trials_df.select(columns).show(5)

+----------------+-------+-------+---------+---------+-----------------+-------------------+------------------------------+-------------+------------------------+
|ageofparticipant| branch|   name|     role|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|   conclusion|sideeffectsonparticipant|
+----------------+-------+-------+---------+---------+-----------------+-------------------+------------------------------+-------------+------------------------+
|              19|Ontario|   Saul|therapist|  Placebo|    1619827200000|      1617235200000|                            52|BP normalized|          rashes on neck|
|              14|Ontario|   Saul|    nurse| Naproxen|    1619827200000|      1617235200000|                            78|    Follow-up|                     N/A|
|              17|Ontario|   Saul|    nurse|  Placebo|    1619827200000|      1617235200000|                            14|    Follow-up|                     N/A|
|              18|Onta

In [18]:
from pyspark.sql import functions as fn

In [19]:
#counting null values per column
trials_df.select([fn.count(fn.when(fn.col(column).isNull(), column)).alias(column) for column in columns]).show()

+----------------+----------------+--------------+--------------+---------+-----------------+-------------------+------------------------------+-----------------+-------------------------------+
|ageofparticipant|clinician.branch|clinician.name|clinician.role|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|result.conclusion|result.sideeffectsonparticipant|
+----------------+----------------+--------------+--------------+---------+-----------------+-------------------+------------------------------+-----------------+-------------------------------+
|               0|               0|             0|           109|        0|                0|                  0|                            73|               53|                              0|
+----------------+----------------+--------------+--------------+---------+-----------------+-------------------+------------------------------+-----------------+-------------------------------+



# Cleaning

In [20]:
trials_df.show(5)

+----------------+--------------------+---------+-----------------+-------------------+------------------------------+--------------------+
|ageofparticipant|           clinician|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|              result|
+----------------+--------------------+---------+-----------------+-------------------+------------------------------+--------------------+
|              19|{Ontario, Saul, t...|  Placebo|    1619827200000|      1617235200000|                            52|{BP normalized, r...|
|              14|{Ontario, Saul, n...| Naproxen|    1619827200000|      1617235200000|                            78|    {Follow-up, N/A}|
|              17|{Ontario, Saul, n...|  Placebo|    1619827200000|      1617235200000|                            14|    {Follow-up, N/A}|
|              18|{Ontario, Will, n...| Naproxen|    1619827200000|      1617235200000|                            14|{BP normalized, N/A}|
|              17|{O

In [ ]:
# flatten the dataframe
# address null values
# rename the columns
#

In [21]:
new_trials_df = trials_df.select(columns)

In [22]:
new_trials_df.printSchema()

root
 |-- ageofparticipant: long (nullable = true)
 |-- branch: string (nullable = true)
 |-- name: string (nullable = true)
 |-- role: string (nullable = true)
 |-- drug_used: string (nullable = true)
 |-- experimentenddate: string (nullable = true)
 |-- experimentstartdate: string (nullable = true)
 |-- noofhourspassedatfirstreaction: long (nullable = true)
 |-- conclusion: string (nullable = true)
 |-- sideeffectsonparticipant: string (nullable = true)



In [26]:
# rename the column
new_trials_df = new_trials_df.withColumnsRenamed({'sideeffectsonparticipant': 'side effects', 'noofhourspassedatfirstreaction': 'no of hours passed at first reaction', 'experimentstartdate': 'experiment start date', 'ageofparticipant': 'age of participant', 'experimentenddate': 'experiment end date', 'name': 'head clinician', 'role': 'assistant role'})

In [27]:
new_trials_df.show()

+------------------+-------+--------------+--------------+---------+-------------------+---------------------+------------------------------------+-------------+--------------+
|age of participant| branch|head clinician|assistant role|drug_used|experiment end date|experiment start date|no of hours passed at first reaction|   conclusion|  side effects|
+------------------+-------+--------------+--------------+---------+-------------------+---------------------+------------------------------------+-------------+--------------+
|                19|Ontario|          Saul|     therapist|  Placebo|      1619827200000|        1617235200000|                                  52|BP normalized|rashes on neck|
|                14|Ontario|          Saul|         nurse| Naproxen|      1619827200000|        1617235200000|                                  78|    Follow-up|           N/A|
|                17|Ontario|          Saul|         nurse|  Placebo|      1619827200000|        1617235200000|     

In [29]:
new_trials_df.describe().show()

23/09/02 11:52:20 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+-------+--------------+--------------+---------+--------------------+---------------------+------------------------------------+-------------+--------------+
|summary|age of participant| branch|head clinician|assistant role|drug_used| experiment end date|experiment start date|no of hours passed at first reaction|   conclusion|  side effects|
+-------+------------------+-------+--------------+--------------+---------+--------------------+---------------------+------------------------------------+-------------+--------------+
|  count|              3586|   3586|          3586|          3477|     3586|                3586|                 3586|                                3513|         3533|          3586|
|   mean|17.507250418293363|   null|          null|          null|     null|1.618381578137200...| 1.615813671834913...|                   44.89097637346997|         null|          null|
| stddev|2.3066401927555233|   null|          null|          null|    

In [34]:
new_trials_df = new_trials_df.na.fill({'conclusion': 'unknown', 'assistant role': 'unknown'})

In [35]:
new_trials_df.describe().show()

+-------+------------------+-------+--------------+--------------+---------+--------------------+---------------------+------------------------------------+-------------+--------------+
|summary|age of participant| branch|head clinician|assistant role|drug_used| experiment end date|experiment start date|no of hours passed at first reaction|   conclusion|  side effects|
+-------+------------------+-------+--------------+--------------+---------+--------------------+---------------------+------------------------------------+-------------+--------------+
|  count|              3586|   3586|          3586|          3586|     3586|                3586|                 3586|                                3513|         3586|          3586|
|   mean|17.507250418293363|   null|          null|          null|     null|1.618381578137200...| 1.615813671834913...|                   44.89097637346997|         null|          null|
| stddev|2.3066401927555233|   null|          null|          null|    